In [129]:
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.datasets.samples_generator import make_blobs
import numpy as np
import h5py

In [130]:
## Generate data
X, y = make_blobs(n_samples=100, centers=3, n_features=2)

In [127]:
with h5py.File('data/usps.h5', 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]

In [148]:
## Compute RBF matrix K
K = rbf_kernel(X_tr)

In [132]:
def transfer_func(x, kernel='linear', cut=None, t=None):
    if kernel == 'linear':
        return x
    if kernel == 'step':
        if x >= cut:
            return 1
        else:
            return 0
    if kernel == 'lin-step':
        if x >= cut:
            return x
        else:
            return 0
    if kernel == 'poly':
        return x**t

In [133]:
## 1. Compute D, the diagonal matrix whose elements are the sum of the rows of K
D = np.zeros(np.shape(K))
count = 1
for i in range(np.shape(K)[0]):
    D[i, i] = np.sum(K[i, :])

In [143]:
## 2. Compute L
L = np.linalg.inv(np.sqrt(D)) @ K @ np.linalg.inv(np.sqrt(D))
eigenvals, eigenvects = np.linalg.eig(L)
Lambda = np.diag(eigenvals)

In [144]:
## 3. Given transfer function, construct L_hat
new_eigenvals = np.zeros(np.shape(eigenvals)) # lamdba_hat_i

for i in range(np.shape(eigenvals)[0]):
    new_eigenvals[i] = transfer_func(eigenvals[i], kernel='linear', t=5)
    
lambda_hat = np.diag(new_eigenvals)
L_hat = eigenvects @ lambda_hat @ eigenvects.T

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: ComplexWarning: Casting complex values to real discards the imaginary part
  """


KeyboardInterrupt: 

In [ ]:
## 4. Compute K_hat
D_hat = np.linalg.inv(L_hat)
K_hat = np.sqrt(D_hat) @ L_hat @ np.sqrt(D_hat)
K_hat